<a href="https://colab.research.google.com/github/buggytanmoy77/Chaos/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation of necessary libraries and tools

In [ ]:
!pip install -q diffusers transformers accelerate torch xformers
!pip install -q clip-interrogator safetensors huggingface_hub
!pip install -q git+https://github.com/huggingface/transformers.git

## Setting up CLIP

In [ ]:
!pip install --upgrade torchvision
!pip install clip-interrogator==0.6.0

from PIL import Image
from clip_interrogator import Config, Interrogator

ci = Interrogator(Config(
    clip_model_name="ViT-L-14/openai",
    device="cuda"
))

def get_prompt(image_path):
    image = Image.open(image_path).convert("RGB")
    return ci.interrogate(image)

## Download and Initialization of SD model

In [ ]:
!wget https://civitai.com/api/download/models/128713 -O custom_model.safetensors

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_single_file(
    "custom_model.safetensors",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

## Extracting Prompts from Image Files

In [ ]:
image_paths = [
    "/content/test_img1.jpg",
    "/content/test_img2.jpg"
]

prompts = [get_prompt(path) for path in image_paths]
print("\nExtracted Prompts:", prompts)

## Combining and Summarizing Image Prompts Using Transformers

In [7]:
!pip install transformers
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
combined_prompts = " AND ".join([
    f"({prompt})" for prompt in prompts
])

summary = summarizer(combined_prompts, max_length=72, min_length=30, do_sample=False)[0]['summary_text']
summary = summary + ", best quality, high resolution,"
print("Combined and Summarized Prompt:")
print(summary)

Device set to use cuda:0


Combined and Summarized Prompt:
Images include a man in a white robe standing in a corridor, arches adorned pillars, on an indian street, royal palace, awnings, inspired by D. Howard Hitchcock., best quality, high resolution,
